In [1]:
from bitcoincli import Bitcoin
import datetime
import pandas as pd

In [2]:
# creds

host = "172.33.0.2"
port = "8332"
username = "dappnode"
password = "dappnode"

bitcoin = Bitcoin(username, password, host, port)

info = bitcoin.getblockchaininfo()
print(info)

{'chain': 'main', 'blocks': 845404, 'headers': 845404, 'bestblockhash': '000000000000000000022eff93bc98b5c60c6c28f19f4c011ade45ce3fca9169', 'difficulty': 84381461788831.34, 'time': 1716835391, 'mediantime': 1716830061, 'verificationprogress': 0.9999957303936676, 'initialblockdownload': False, 'chainwork': '00000000000000000000000000000000000000007ba6860b3bd4fc36f3c9823e', 'size_on_disk': 652887916389, 'pruned': False, 'warnings': ''}


In [3]:
def get_latest_block():
    return bitcoin.getblockcount()

In [4]:
bheight = 830000
print("bheight", bheight)
bhash = bitcoin.getblockhash(bheight)

print("bhash", bhash)

bheight 830000
bhash 000000000000000000011d55599ed27d7efca05f5849b755319c89eb2cffbc1f


In [5]:
def get_tx(api, txid, blockhash):
    return api.getrawtransaction(txid, True, blockhash)

In [6]:
block_info = bitcoin.getblock(bhash)

In [7]:
txid = block_info['tx'][1]

In [8]:
txinfo = get_tx(bitcoin, txid, bhash)

In [9]:
import time

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Время выполнения функции '{func.__name__}': {execution_time:.6f} секунд")
        print(f"Аргументы: args={args}, kwargs={kwargs}")
        return result
    return wrapper

In [10]:
def ts_to_datetime(ts):
    return datetime.datetime.utcfromtimestamp(ts)


def block_info_to_datetime(block_info):
    return ts_to_datetime(block_info['time'])

@timing_decorator
def binary_search(getfn, target, left, right):
    while right - left > 1:
        middle = (left + right) // 2
        if getfn(middle) >= target:
            right = middle
        else:
            left = middle
    return left

In [11]:
ts_to_datetime(block_info['time'])

datetime.datetime(2024, 2, 11, 18, 17, 37)

In [12]:
dt_start = datetime.datetime(2024, 3, 13, 22, 59, 0)
dt_end = datetime.datetime(2024, 3, 18, 18, 34, 0)

In [13]:
def get_block_by_height(api, height):
    blockhash = api.getblockhash(height)
    return api.getblock(blockhash)

In [14]:
def block_date_by_height(height):
    block_info = get_block_by_height(bitcoin, height)
    return block_info_to_datetime(block_info)

In [15]:
search_start = 830000
search_end = 840000
print(block_date_by_height(search_start))
print(block_date_by_height(search_end))

2024-02-11 18:17:37
2024-04-20 00:09:27


In [16]:
start_block = binary_search(block_date_by_height, dt_start, search_start, search_end)
start_block

Время выполнения функции 'binary_search': 2.232220 секунд
Аргументы: args=(<function block_date_by_height at 0x7f54200abeb0>, datetime.datetime(2024, 3, 13, 22, 59), 830000, 840000), kwargs={}


834577

In [17]:
end_block = binary_search(block_date_by_height, dt_end, search_start, search_end)
end_block = end_block + 1
end_block

Время выполнения функции 'binary_search': 2.369087 секунд
Аргументы: args=(<function block_date_by_height at 0x7f54200abeb0>, datetime.datetime(2024, 3, 18, 18, 34), 830000, 840000), kwargs={}


835257

In [18]:
end_block - start_block

680

In [19]:
print(block_date_by_height(start_block))
print(block_date_by_height(end_block))

2024-03-13 22:40:42
2024-03-18 18:36:42


In [20]:
block_info.keys()

dict_keys(['hash', 'confirmations', 'height', 'version', 'versionHex', 'merkleroot', 'time', 'mediantime', 'nonce', 'bits', 'difficulty', 'chainwork', 'nTx', 'previousblockhash', 'nextblockhash', 'strippedsize', 'size', 'weight', 'tx'])

In [21]:
@timing_decorator
def get_block_txs(height):
    txs = []
    block_info = get_block_by_height(bitcoin, height)
    block_hash = block_info['hash']
    block_txs = block_info['tx']
    for txid in block_txs:
        txs.append(get_tx(bitcoin, txid, block_hash))
    return txs

In [22]:
start_block_txs = get_block_txs(start_block)

Время выполнения функции 'get_block_txs': 48.004380 секунд
Аргументы: args=(834577,), kwargs={}


In [23]:
len(start_block_txs)

1362

In [24]:
df = pd.DataFrame(start_block_txs)
df.to_csv('rpc_block_834577_txs.csv', index=False)

In [ ]:
txs = []
for bn in range(start_block, end_block):
    txs.extend(get_block_txs(bn))
fulldf = pd.DataFrame(txs)
fulldf.to_csv('rpc_blocks.csv', index=False)

Время выполнения функции 'get_block_txs': 42.791564 секунд
Аргументы: args=(834577,), kwargs={}
Время выполнения функции 'get_block_txs': 99.378232 секунд
Аргументы: args=(834578,), kwargs={}
Время выполнения функции 'get_block_txs': 69.151426 секунд
Аргументы: args=(834579,), kwargs={}
Время выполнения функции 'get_block_txs': 62.915584 секунд
Аргументы: args=(834580,), kwargs={}
Время выполнения функции 'get_block_txs': 72.019788 секунд
Аргументы: args=(834581,), kwargs={}
Время выполнения функции 'get_block_txs': 75.943256 секунд
Аргументы: args=(834582,), kwargs={}
Время выполнения функции 'get_block_txs': 74.574356 секунд
Аргументы: args=(834583,), kwargs={}
Время выполнения функции 'get_block_txs': 49.891699 секунд
Аргументы: args=(834584,), kwargs={}
Время выполнения функции 'get_block_txs': 35.135821 секунд
Аргументы: args=(834585,), kwargs={}
Время выполнения функции 'get_block_txs': 77.445652 секунд
Аргументы: args=(834586,), kwargs={}
Время выполнения функции 'get_block_txs'

In [2]:
import pandas as pd
fulldf = pd.read_csv('script_rpc_blocks.csv')

In [3]:
txs = fulldf.to_dict('records')

In [5]:
len(txs)

38580

In [8]:
txs[0].keys()

dict_keys(['in_active_chain', 'txid', 'hash', 'version', 'size', 'vsize', 'weight', 'locktime', 'vin', 'vout', 'hex', 'blockhash', 'confirmations', 'time', 'blocktime'])